In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install gradio faiss-cpu torch torchvision


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [ ]:
import zipfile, os, glob, torch

# STEP 1: Unzip your uploaded dataset from Drive
zip_path = "/content/drive/MyDrive/Flickr8k_Dataset.zip"
unzip_dir = "/content/images"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print("✅ Images extracted to:", unzip_dir)


✅ Images extracted to: /content/images


In [ ]:
'''
# STEP 2: Get embedding files
image_embedding_paths = sorted(glob.glob("/content/image_embeddings_batch_*.pt"))
text_embedding_paths = sorted(glob.glob("/content/text_embeddings_batch_*.pt"))
print(f"✅ Found {len(image_embedding_paths)} image embedding files")
print(f"✅ Found {len(text_embedding_paths)} text embedding files")

# STEP 3: Count total number of embeddings
total_embeddings = 0
for path in image_embedding_paths:
    emb = torch.load(path)
    total_embeddings += emb.shape[0]
print(f"🧠 Total image embeddings available: {total_embeddings}")

# STEP 4: Load images in original order (not sorted)
ordered_files = []
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for file in zip_ref.namelist():
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            ordered_files.append(os.path.join(unzip_dir, file))
original_image_files = [f for f in ordered_files if os.path.exists(f)]
original_image_files = original_image_files[:5000]
print(f"🖼️ Loaded {len(original_image_files)} images to match {total_embeddings} embeddings")
'''
# STEP 4: Load images in original order (not sorted)
ordered_files = []
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    for file in zip_ref.namelist():
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            ordered_files.append(os.path.join(unzip_dir, file))

# Filter only existing files
original_image_files = [f for f in ordered_files if os.path.exists(f)]

# Trim to match the number of indexed embeddings (FAISS index size)
original_image_files = original_image_files[:index.ntotal]
print(f"🖼️ Trimmed image list to match FAISS index: {len(original_image_files)} images")


🖼️ Trimmed image list to match FAISS index: 5000 images


In [ ]:
import faiss

def load_embeddings(paths, limit=5000):
    embeddings = []
    for path in paths:
        emb = torch.load(path)
        if emb.numel() > 0:
            embeddings.append(emb)
        if sum(e.shape[0] for e in embeddings) >= limit:
            break
    return torch.cat(embeddings)[:limit]

image_embeddings = load_embeddings(image_embedding_paths)
text_embeddings = load_embeddings(text_embedding_paths)

index = faiss.IndexFlatL2(512)
index.add(image_embeddings.numpy())


In [ ]:
from transformers import CLIPProcessor, CLIPModel
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [ ]:
import gradio as gr
from PIL import Image
import numpy as np

def search_by_text(text):
    if len(original_image_files) != index.ntotal:
        return [], f"⚠️ Mismatch: {len(original_image_files)} files vs {index.ntotal} index"

    inputs = processor(text=[text], return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        query_embedding = model.get_text_features(**inputs).cpu().numpy()
    D, I = index.search(query_embedding, k=10)

    result_images, result_info = [], []
    for j, (i, d) in enumerate(zip(I[0], D[0])):
        if i < len(original_image_files):
            result_images.append(original_image_files[i])
            result_info.append(f"Match {j+1}: Index {i}, Score {round(float(d), 2)}")
        else:
            result_info.append(f"⚠️ Invalid index: {i}")
    return result_images, "\n".join(result_info)

def search_by_image(img):
    if len(original_image_files) != index.ntotal:
        return [], f"⚠️ Mismatch: {len(original_image_files)} files vs {index.ntotal} index"

    inputs = processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        query_embedding = model.get_image_features(**inputs).cpu().numpy()
    D, I = index.search(query_embedding, k=10)

    result_images, result_info = [], []
    for j, (i, d) in enumerate(zip(I[0], D[0])):
        if i < len(original_image_files):
            result_images.append(original_image_files[i])
            result_info.append(f"Match {j+1}: Index {i}, Score {round(float(d), 2)}")
        else:
            result_info.append(f"⚠️ Invalid index: {i}")
    return result_images, "\n".join(result_info)

with gr.Blocks() as demo:
    gr.Markdown("## 🔍 CLIP-based Visual Search Engine")
    with gr.Tab("Search by Text"):
        txt_input = gr.Textbox(label="Describe an image (e.g., 'a dog running')")
        txt_gallery = gr.Gallery(label="Top Matches", columns=2)
        txt_result_info = gr.Textbox(label="Matched Indices & Scores")
        txt_btn = gr.Button("Search")
        txt_btn.click(fn=search_by_text, inputs=txt_input, outputs=[txt_gallery, txt_result_info])

    with gr.Tab("Search by Image"):
        img_input = gr.Image(type="pil", label="Upload an image", interactive=True)
        img_gallery = gr.Gallery(label="Top Matches", columns=2)
        img_result_info = gr.Textbox(label="Matched Indices & Scores")
        img_btn = gr.Button("Search")
        img_btn.click(fn=search_by_image, inputs=img_input, outputs=[img_gallery, img_result_info])

demo.launch(share=True, debug=True, allowed_paths=["/content/images"])


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ddfe5a2f6ea0ce73ef.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ddfe5a2f6ea0ce73ef.gradio.live
